# ECEN 4322-5322 Data and Network Science

## Title- Fare prediction for flights


#### Group Members - Chirag Chandrashekar, Chris Alexander, Viveka Salinamekki.

## Introduction 

The dataset chosen for analysis is the itineraries of flights in the USA over 6 months. For exploratory data analysis, we aim to find the airport with the highest traffic or the best-connected airports, the price of flights during the different hours of the day, the average/minimum travel distance for which people prefer flight, whether an average flight is fully booked/percentage of flights booked, and popular airlines. The end goal here is to predict the fare of a flight. Due to the large number of rows and features, the prediction can provide a good estimation of the fare. Techniques such as data grouping and manipulation, visualization, regular expressions, data modeling, feature engineering, model validation, and prediction will help achieve our goal. 


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Importing the Data

In [2]:
import numpy as np

import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import zipfile
import os

from ecen5322_utils import run_linear_regression_test

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

### Sampling the dataset

Write how the data is read as chunks and sampled

In [ ]:
'''
dfs=[]
with pd.read_csv("car_price_prediction.csv", chunksize=1000000) as reader: #read as chunks due to low system resorces to read 30gb file
    reader
    for chunk in reader:
        #print(type(chunk))
        data_index=chunk.index
        shuffled_indices = np.random.permutation(data_index)  #shuffling and sampling data to reduce the data to 500000 entries
        #print(chunk.loc[shuffled_indices])
        #print(type(chunk))
        chunk,leftover= np.split(chunk.loc[shuffled_indices],[12000]) #selects first N rows from each chunk
        #print(type(chunk))
        #print(chunk)
        #chunks=chunk.to_frame
        #print(chunks)
        dfs.append(chunk) #makes a list of dataframe chunks
        #joined_chunk=pd.concat(chunk)
        #print(joined_chunk)
final_df=pd.concat(dfs) #concats all the chunks in the list of dataframe
print("final dataframe")
final_df
'''

In [ ]:
#Load data to new csv file
#final_df
'''
data_index2=final_df.index
shuffled_indices2 = np.random.permutation(data_index2)
final_df2,leftover2=np.split(final_df.loc[shuffled_indices2],[500000])
final_df2
'''

In [ ]:
#from google.colab import  files
#final_df2.to_csv('sampled_file.csv')
#files.download('sampled_file.csv')

In [49]:
#data=pd.read_csv('sampled_file.csv')

#New read_csv function to read empty values as -1 and remove unamed column
data=pd.read_csv("sampled_file.csv"
                 #, dtype=str
                 , keep_default_na=True
                 , na_values=-1
                 , na_filter=True).drop(columns=['Unnamed: 0'])
data

False

In [47]:
#analysis of Chris's columns
Chris_columns=data.iloc[:,9:19]
print(Chris_columns)
specific_col= Chris_columns.iloc[:,9]
specific_col.value_counts()
#.value_counts().sort_values()
#.sort_values()

        isRefundable  isNonStop  baseFare  totalFare  seatsRemaining  \
0              False      False    160.93     196.60               7   
1              False      False    294.89     349.21               3   
2              False      False    430.70     486.60               1   
3              False       True    189.77     218.60               9   
4              False      False    212.09     257.20               9   
...              ...        ...       ...        ...             ...   
499995         False      False    488.37     548.60               7   
499996         False      False    373.95     434.20               7   
499997         False      False    579.53     651.10               9   
499998         False      False    310.70     357.60               3   
499999         False      False    659.54     737.11               3   

        totalTravelDistance   segmentsDepartureTimeEpochSeconds  \
0                     465.0              1655304180||1655318280   
1

2022-07-26T14:40:00.000-04:00                                                                  10
2022-08-23T15:21:00.000-04:00                                                                  10
2022-07-21T14:40:00.000-04:00                                                                   9
2022-08-23T23:30:00.000-04:00                                                                   9
2022-08-27T16:30:00.000-04:00                                                                   8
                                                                                               ..
2022-05-25T10:58:00.000-04:00||2022-05-25T15:56:00.000-05:00                                    1
2022-10-04T07:46:00.000-04:00||2022-10-04T11:19:00.000-05:00                                    1
2022-08-02T21:20:00.000-04:00||2022-08-02T23:50:00.000-04:00                                    1
2022-07-22T08:57:00.000-04:00||2022-07-22T13:50:00.000-05:00||2022-07-22T16:57:00.000-07:00     1
2022-09-16T09:46:00.

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Cleaning the data

### make NaN =0  

### convert True and false to 0 and 1

### remove base fare less than 50

### rename columns

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Exploratory Data Analysis

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Prediction of____

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Validation of model

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Inference and Conclusion